In [ ]:
import json
print('json', json.__version__)

In [ ]:
from collections import OrderedDict

In [ ]:
from functools import reduce

In [ ]:
import pandas as pd
print("Pandas", pd.__version__)

In [ ]:
import numpy as np
print('numpy', np.__version__)

In [ ]:
import matplotlib
print("matplotlib", matplotlib.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
%matplotlib inline

In [ ]:
import manager

In [ ]:
cfg = json.load(open('config.json'))
print('config', cfg)

In [ ]:
in_folder   = cfg['in_folder']

In [ ]:
db_folder   = cfg['db_folder']

In [ ]:
db_date_file_pairs = manager.get_db_date_files(db_folder, in_folder)
# db_date_file_pairs

In [ ]:
assert len(db_date_file_pairs) > 0, "NO FILES FOUND"

In [ ]:
#print( "\n".join( [ "{:3d}/{:3d} {}".format(fn+1, len(db_date_file_pairs), f) for fn, (d,f) in enumerate(db_date_file_pairs.items())if f != '' ] ) )

In [ ]:
describes = OrderedDict()
all_data = None

for fn, (d,f) in enumerate(db_date_file_pairs.items()):
    print("{:3d}/{:3d} {} {}".format(fn+1, len(db_date_file_pairs), d, f))
    df             = pd.read_excel(f)
#     print(df)
    dsc = df.describe().transpose()
    all_data = pd.concat([all_data, df.transpose()], axis=1, join_axes=[all_data.index]) if all_data is not None else df.transpose()
#     print(dsc)
    describes[d] = [ f, dsc ]
    if fn ==3: break

In [ ]:
all_data_new = pd.DataFrame(all_data.transpose().to_dict())
all_data_new = all_data_new.set_index('GPS Time')
all_data_new

In [ ]:
all_data_new.describe()

In [ ]:
figure(figsize=(20,5))
all_data_new.boxplot()

In [ ]:
for col in all_data_new.columns:
    print(col)
    ad = all_data_new[[col]].dropna()
#     print(ad, ad.columns, ad.index)
    figure(figsize=(20,5))
    ad.boxplot()
    plt.show()

In [ ]:
for col in all_data_new.columns:
    print(col)
    ad = all_data_new[[col]].dropna()
#     print(ad, ad.columns, ad.index)
    
    if len(ad.index) == 0:
        continue
        
    figure(figsize=(20,5))
    try:
        ad.hist()
        plt.show()
    except ValueError:
        pass


In [ ]:
#https://machinelearningmastery.com/quick-and-dirty-data-analysis-with-pandas/
from pandas.plotting import scatter_matrix
scatter_matrix(all_data_new, alpha=0.2, figsize=(20, 20), diagonal='kde')

In [ ]:
all_data_new.to_csv('all_data.csv')

In [ ]:
sfx  = list(k for k in describes.keys())
vals = [ v[1] for v in describes.values() ]
# print(sfx)
# print(vals)
descriptions = pd.concat(vals, keys=sfx).transpose()
descriptions

In [ ]:
descriptions.to_csv('stats.csv')

In [ ]:
# stats = sorted(list(set([ l2 for l1, l2 in descriptions.columns ])))
stats = sorted(list(set([str(v) for v in descriptions.columns.get_level_values(1)])))
print(stats)

In [ ]:
indexes = list(descriptions.index)
indexes

In [ ]:
def bar_plotter(title, df):
    cols = df.columns.get_level_values(0)
    num_cols = len(cols)
#     print(num_cols)
    means = df.loc['mean']
    std = df.loc['std']
    mins = df.loc['min']
    maxes = df.loc['max']
    figure(figsize=(20,5))
    plt.title(title)
    plt.errorbar(cols, means, std, fmt='ok', lw=3)
    plt.errorbar(cols, means, [means - mins, maxes - means],
                 fmt='.k', ecolor='gray', lw=1)
#     plt.xlim(-1, num_cols)

In [ ]:
plotters = OrderedDict((
#     (('count',), lambda x, y: y.plot.bar(title=x) ),
    (('mean','std','min','max'), bar_plotter ),
))
print(plotters)

In [ ]:
for stat in stats:
#     print(stat)

    sc = descriptions.columns.get_level_values(1)==stat
#     print(sc, sum(sc))

    sd = descriptions.iloc[:,sc]
#     print(sd)
    if len(sd.columns) == 0:
        continue
    
    for cols, plot_type in plotters.items():
        cols = list(cols)
#         print(' cols', cols)#, 'plot_type', plot_type)
        sds = sd.loc[cols]
#         sds.columns
        sds.columns = sds.columns.droplevel(1)
#         print(sds)
        plot_type(stat, sds)
        plt.show()
#     break